In [16]:
#Для решения заданий будут необходимы следующие пакеты:

import pandas as pd
import numpy as np
import surprise
import apyori

In [ ]:
#Если библиотеки surprise и apyori не установлены, воспользуйтесь менеджером пакетов pip:

! pip install surprise
! pip install apyori

In [26]:
#И датасет из библиотеки surprise:

data = surprise.Dataset.load_builtin("ml-100k")

trainingSet = data.build_full_trainset()
testSet = trainingSet.build_testset()

#### Задание 9.10.1

Найдите количество пользователей в trainingSet 

In [27]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user_id', 'movie_id', 'rating', 3]
df.user_id.nunique()
#943 is correct

943

#### Задание 9.10.2
Найдите количество items в trainingSet

In [25]:
df.movie_id.nunique()
#1682 is correct

1682

#### Задание 9.10.3
Проведите тренировку алгоритма user-based коллаборативной фильтрации KNNBasic и алгоритма SVD.

Постройте рекомендации testset c использованием двух алгоритмов, натренированных выше. Выведите предсказание под 8-ым индексом. В качестве ответ укажите id рекомендованного фильма.

In [38]:
#https://surprise.readthedocs.io/en/stable/knn_inspired.html
from surprise import KNNBasic

# 'name' : The name of the similarity to use, as defined in the similarities module.
sim_options = {'name': 'cosine','user_based': True} #Compute the cosine similarity between all pairs of users (or items).

knn = KNNBasic(sim_options=sim_options)

knn.fit(trainingSet)
knn_predictions = knn.test(testSet)
knn_predictions[8]  

Computing the cosine similarity matrix...
Done computing similarity matrix.


Prediction(uid='196', iid='663', r_ui=5.0, est=4.475491686108841, details={'actual_k': 40, 'was_impossible': False})

In [39]:
from surprise import SVD

#Задаём наш алгоритм и формируем обучающую выборку:
svd=SVD()
trainingSet=data.build_full_trainset()

#Тренируем наш алгоритм:
svd.fit(trainingSet)

#Далее мы уже можем получить предсказание:
testSet=trainingSet.build_testset()
svd_predictions=svd.test(testSet)
svd_predictions[8]

Prediction(uid='196', iid='663', r_ui=5.0, est=4.329099810705593, details={'was_impossible': False})

In [40]:
#663 is correct
#iid - film id

#### Задание 9.10.4
Рассчитайте метрику качества RMSE для предсказаний через функцию surprise.accuracy.rmse() для обоих обученных ранее алгоритмов. В качестве ответа укажите значение RMSE для того алгоритма, который дал лучший результат. Округлите ответ до двух знаков после точки-разделителя.

In [46]:
sar_knn = surprise.accuracy.rmse(knn_predictions, verbose=False)
sar_svd = surprise.accuracy.rmse(svd_predictions, verbose=False)

print(f'knn_predictions = {sar_knn}') 
print(f'svd_predictions = {sar_svd}')
print(f'RMSE for better predictions = {np.round(min(sar_knn, sar_svd), 2)}') #Lower values of RMSE indicate better fit.
#0.68 is correct

knn_predictions = 0.9012175824620821
svd_predictions = 0.6750535440409582
RMSE for better predictions = 0.68


In [103]:
#Cформируем привычный DataFrame из нашего датасета

df = pd.DataFrame(data.raw_ratings)
df.columns = ['user_id', 'movie_id', 'rating', 3]
df.head()

,user_id,movie_id,rating,3
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


#### Задание 9.10.5
Сформируйте для каждого user_id текстовую строку, содержащую набор movie_id, получивших оценку 4 и выше. Используйте пробел (" ") в качестве разделителя.

Для проверки результата введите строку для user_id 100 (имелась ввиду 100 строка, если попробовать как-то так ваша_табличка['100'] ). Укажите параметр random_state=829.

Пример ответа: 61 33 160 20 202 171 265 47 222 253 113 227 90...

In [104]:
#v1
df_movies = pd.DataFrame(df[df['rating'] > 4].groupby(['user_id'])['movie_id'].apply(lambda x: ' '.join(list(x))))
df_movies.head()

,movie_id
user_id,
1,202 171 253 113 64 228 114 221 60 177 174 82 2...
10,611 100 488 285 504 655 124 22 656 200 223 135...
100,316 313 315
101,237
103,126 527 50 255 56


In [105]:
#v2
#Из полученного датасета возьмём только те записи, у которых наивысший рейтинг > 4 и объединим их по "user_id". 
#Фильмы сгруппируем в строчку с разделителем "пробел" так, чтобы для каждого пользователя была строка с ID тех фильмов, которые ему понравились:
good = df[df['rating'] > 4].groupby('user_id')['movie_id'].apply(lambda r: ' '.join([str(A) for A in r]))
good[100]

'302 471 313 272 237 288'

In [102]:
df_movies

,movie_id
user_id,
1,202 171 253 113 64 228 114 221 60 177 174 82 2...
10,611 100 488 285 504 655 124 22 656 200 223 135...
100,316 313 315
101,237
103,126 527 50 255 56
...,...
95,1 174 173 471 14 83 204 660 64 404 657 747 7 6...
96,91 8 484 645 64 100 170 483 1 1232 50 23 1154 ...
97,228 222 670 89 482 50 115 7 28 655 97 79 430 3...


In [ ]:
data = surprise.Dataset.load_builtin("ml-100k")
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user_id', 'movie_id', 'rating', 3]
df_movies = pd.DataFrame(df[df['rating'] > 3].groupby(['user_id'])['movie_id'].apply(lambda x: ' '.join(list(x))))
association_rules = apyori.apriori(df_movies, 
                                   min_support=0.2, 
                                   min_confidence=0.3, min_lift=2, 
                                   min_length=2)

#### Задание 9.10.6
Используя алгоритм apyori из модуля apyori, рассчитайте ассоциативные правила для полученного набора фильмов с положительными оценками. Будем считать, что хороший рейтинг — это когда оценка >= 4. Используйте следующие ограничения:

    минимальный Support 0.2;
    минимальное значение Confidence 0.3;
    минимальное значение Lift 2;
    минимальная длина 2.

Сколько правил получилось? 

#### Задание 9.10.7
Какой фильм чаще всего встречается в ассоциативных правилах? Укажите id фильма.